## Add Custom Narrative to FHIR Search Parameter

- Get SP from file
- Generate Narrative from Jinja2 Template
- Strip out any null values
- Save SP back to file

In [1]:
from json import dumps, loads, load
from pathlib import Path
from IPython.display import display as Display, HTML, Markdown
from jinja2 import Environment, FileSystemLoader, select_autoescape
from lxml import etree
from fhirclient.r4models import narrative as N
from commonmark import commonmark

### Get SP from file

In [2]:
in_path = Path() / 'test' / 'in_sp.json' # temp path
sp = loads(
        in_path.read_text()
    )

### Generate Narrative from Jinja2 Template

In [3]:
def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = ''),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )


env.filters['markdown'] = markdown

template = env.get_template('sp_narrative.j2')  

rendered = template.render(sp=sp)
display(HTML(rendered))


parser = etree.XMLParser(remove_blank_text=True)
root = etree.fromstring(rendered, parser=parser)

div = (etree.tostring(root[1][0], encoding='unicode', method='html'))

narr = N.Narrative()
narr.status = 'generated'
narr.div = div

sp['text'] = narr.as_json()

### Save SP back to file

In [4]:
out_path = Path() / 'test' / 'out_sp.json' # temp path
out_path.write_text(dumps(sp, indent = 2))

3722